In [1]:
from transformers import AutoModelForMaskedLM
from adapters import init, AutoAdapterModel
model_checkpoint = "bert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
init(model)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of the model checkpoint at bert-base-uncased were not used when initi

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload

['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/tmp/tmp6302e46g', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


2024-03-21 02:23:54.823713: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 02:23:54.887556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 02:23:55.482799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
fn.print_trainable_parameters(model)

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [4]:
model.delete_adapter("task_adapter")

In [5]:
adapter_name="mlm_adapter"
model.add_adapter(adapter_name, config="seq_bn")
fn.print_trainable_parameters(model)

trainable params: 110408826 || all params: 110408826 || trainable%: 100.0


In [6]:

model.set_active_adapters(adapter_name)
model.train_adapter(adapter_name)
fn.print_trainable_parameters(model)
model.active_adapters

trainable params: 1486656 || all params: 110408826 || trainable%: 1.346501048747679


Stack[mlm_adapter]

In [7]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttentionWithAdapters(
              (query): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (value): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (dro

In [8]:
text = "This is a great [MASK]."

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great idea.'
'>>> This is a great day.'
'>>> This is a great surprise.'
'>>> This is a great story.'
'>>> This is a great place.'


In [11]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [12]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [13]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:5]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 363'
'>>> Review 1 length: 304'
'>>> Review 2 length: 133'
'>>> Review 3 length: 185'
'>>> Review 4 length: 495'


In [14]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [15]:
lm_datasets = tokenized_datasets.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [16]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

In [17]:
tokenizer.decode(lm_datasets["train"][1]['labels'])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] sleeps rented i am curious - yellow from my video store because of all the controversy that [MASK] it [MASK] [MASK] was [MASK] released in 1967 [MASK] i also heard that at first it was [MASK] by u. s. customs if it ever tried to enter this country, [MASK] being a [MASK] [MASK] [MASK] considered " controversial " i really had to see this [MASK] myself. [MASK] br / > [MASK] br / > the [MASK] is centered around a young swedish dramairo named lena who wants to learn everything she can aboutmara. synthetic particular she wants [MASK] focus her [MASK] [MASK] to making some sort of drops [MASK] what the average [MASK]ede thought about certain political issues such'

'>>> as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, [MASK] has sex with her drama teacher, classmates, and married men. < br / > [MASK] stumble / > what kills me about i am [MASK] - yellow is [unused562] 40 year

In [19]:
dd

{'input_ids': tensor([[  101, 25126, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
          2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,   103,  2009,
           103,   103,  2001,   103,  2207,  1999,  3476,   103,  1045,  2036,
          2657,  2008,  2012,  2034,  2009,  2001,   103,  2011,  1057,  1012,
          1055,  1012,  8205,  2065,  2009,  2412,  2699,  2000,  4607,  2023,
          2406,  1010,   103,  2108,  1037,   103,   103,   103,  2641,  1000,
          6801,  1000,  1045,  2428,  2018,  2000,  2156,  2023,   103,  2870,
          1012,   103,  7987,  1013,  1028,   103,  7987,  1013,  1028,  1996,
           103,  2003,  8857,  2105,  1037,  2402,  4467,  3689,  9711,  2315,
         14229,  2040,  4122,  2000,  4553,  2673,  2016,  2064,  2055, 28225,
          1012, 12553,  3327,  2016,  4122,   103,  3579,  2014,   103,   103,
          2000,  2437,  2070,  4066,  1997,  9010,   103,  2054,  1996,  2779,
           103, 14728,  2245,  2055,  

In [20]:
reload(fn)
samples = [lm_datasets["train"][i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented i am curious - yellow from my [MASK] store because of [MASK] the controversy that surrounded it when it was first released in 1967. i also [MASK] that [MASK] first it [MASK] seized by u. s [MASK] [MASK] [MASK] it ever tried to enter [MASK] country, [MASK] being a fan of films considered [MASK] controversial " i really had to see this for myself. < br / > < br / > the plot is centered around a young swedish drama student named [MASK] who wants to learn everything she [MASK] about [MASK]. in particular [MASK] wants to focus her attentions [MASK] making [MASK] sort of documentary on what the average swede thought [MASK] certain political issues such'

'>>> as the vietnam [MASK] and race issues in the united states. in between asking politicians and ordinary denizens [MASK] [MASK] about their [MASK] on politics, [MASK] has sex with her drama [MASK], classmates, and married men. < br / > < [MASK] / [MASK] what [MASK] me about [MASK] am curious - yellow is [MASK] 40 year

In [21]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [25]:
from transformers import TrainingArguments

batch_size = 32
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=10,
    
    learning_rate=2e-5,
    weight_decay=0.01,
    remove_unused_columns=False,#for collator word id
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    logging_steps=logging_steps,
)

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=lambda features: fn.whole_word_masking_data_collator(features, tokenizer),
    tokenizer=tokenizer,
)

In [27]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

>>> Perplexity: 31.55


In [28]:
trainer.train()

  0%|          | 0/3130 [00:00<?, ?it/s]

{'loss': 3.4103, 'learning_rate': 1.8012779552715657e-05, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 3.100553035736084, 'eval_runtime': 1.5516, 'eval_samples_per_second': 644.48, 'eval_steps_per_second': 20.623, 'epoch': 1.0}


Checkpoint destination directory bert-base-uncased-finetuned-imdb/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Removed shared tensor {'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


...

Checkpoint destination directory bert-base-uncased-finetuned-imdb/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 3.1044, 'learning_rate': 7.028753993610224e-08, 'epoch': 9.97}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 2.941898822784424, 'eval_runtime': 1.5776, 'eval_samples_per_second': 633.893, 'eval_steps_per_second': 20.285, 'epoch': 10.0}
{'train_runtime': 326.3997, 'train_samples_per_second': 306.373, 'train_steps_per_second': 9.589, 'train_loss': 3.1683096443883145, 'epoch': 10.0}


TrainOutput(global_step=3130, training_loss=3.1683096443883145, metrics={'train_runtime': 326.3997, 'train_samples_per_second': 306.373, 'train_steps_per_second': 9.589, 'train_loss': 3.1683096443883145, 'epoch': 10.0})

In [29]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

>>> Perplexity: 19.84


In [30]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttentionWithAdapters(
              (query): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (value): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (dro

In [31]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [32]:
masked_sentence = "This movie is [MASK]."
predictions = mask_filler(masked_sentence)

# Display the predictions
for prediction in predictions:
    print(prediction)


{'score': 0.1058935597538948, 'token': 2439, 'token_str': 'lost', 'sequence': 'this movie is lost.'}
{'score': 0.05453759431838989, 'token': 2919, 'token_str': 'bad', 'sequence': 'this movie is bad.'}
{'score': 0.030901392921805382, 'token': 4333, 'token_str': 'silent', 'sequence': 'this movie is silent.'}
{'score': 0.02835305966436863, 'token': 2204, 'token_str': 'good', 'sequence': 'this movie is good.'}
{'score': 0.022351818159222603, 'token': 1038, 'token_str': 'b', 'sequence': 'this movie is b.'}


In [33]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> this is a great idea.
>>> this is a great place.
>>> this is a great day.
>>> this is a great movie.
>>> this is a great book.


prexibilty with full fine tuning 4 epoch is :26.43
prexibilty with adapter fine tuning 10 epoch is :30


when ic hanged the collator from word whole masking prexibiilty got better even before training. with whole word it got 60 (47.14 with 15%) before training and with normal collator it got 23 before training and >>> Perplexity: 11.08 after training. This is full tuning.

distillbert with adapter with normal collator:
before: 23, after  13

bert with adapter with normal collator:
before: 14.5, after  10
